In [1]:
%load_ext autoreload

In [2]:
%autoreload
from dotenv import load_dotenv
from src.data import scrape_genius_lyrics as scr
import os

In [3]:
load_dotenv()
GENIUS = os.getenv("GENIUS")

In [6]:
hit = scr.return_top_hit(['kaskade', 'angel on my shoulders'], token=GENIUS)
hit_url = scr.extract_url(hit)
hit_lyrics = scr.extract_lyrics(hit_url)

In [7]:
print(hit_lyrics)



[Verse 1]
It's time you see
The love you give
Is killing me
And I want to live
This isn't right
Search your soul for light

[Pre-Chorus]
So I'm turning left
I told myself
I could be as bad as someone else
But I find it hard
When angel's on my right

[Chorus]
But you held my hand
And took me right back down to hell
I have an angel on my shoulder but a devil in my hand
But you held my hand
And took me right back down to hell
I have an angel on my shoulder but a devil in my hand

[Verse 2]
Why won't you go
No means no
Your dirty life
Is like a one man show
You're a night attack
You turn the light to black

[Pre-Chorus]
So I'm turning left
I told myself
I could be as bad as someone else
But I find it hard
When angel's on my right

[Chorus]
But you held my hand
And took me right back down to hell
I have an angel on my shoulder but a devil in my hand
But you held my hand
And took me right back down to hell
I have an angel on my shoulder but a devil in my hand

[Bridge]
Talk to me
Walk with